# Main

> Sync files between directories

In [ ]:
#| default_exp main

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import os
import shutil
import hashlib
import re
import yaml
from datetime import datetime
from dataclasses import dataclass
from pathlib import Path
from typing import Callable, Dict, Iterable, Optional, Tuple, List, Any


In [ ]:
#| export

CHUNK_SIZE = 1024 * 1024  # 1MB chunks for hashing/IO

@dataclass
class FileMeta:
    size: int
    mtime: float
    hash: Optional[str] = None  # fill if you want content checks

Index = Dict[str, FileMeta]  # key = relative path using Posix-style ('a/b/c.txt')


In [ ]:
#| export

def rel_key(root: Path, p: Path) -> str:
    return str(p.relative_to(root).as_posix())

def file_hash(path: Path, algo: str = "md5") -> str:
    h = hashlib.new(algo)
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(CHUNK_SIZE), b""):
            h.update(chunk)
    return h.hexdigest()

def should_ignore(p: Path, ignore_globs: Iterable[str]) -> bool:
    return any(p.match(glob) for glob in ignore_globs)



In [ ]:
#| export

def parse_quarto(path: str) -> Tuple[Optional[Dict[Any, Any]], str]:
    """
    Parse a Quarto document and extract YAML frontmatter and markdown content.
    """
    file_path = Path(path)
    
    if not file_path.exists():
        raise FileNotFoundError(f"File not found: {path}")

    filename, extension = os.path.splitext(file_path)

    if extension != ".qmd":
        raise ValueError(f"File ({filename}.{extension}) must be a .qmd file")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as f:
            content = f.read()
    
    if not content.strip():
        return None, ""
    
    yaml_pattern = r"^---\s*\n(.*?)\n---\s*\n?(.*)"
    match = re.match(yaml_pattern, content, re.DOTALL)
    
    if match:
        yaml_content, markdown_content = match.groups()
        
        try:
            if yaml_content.strip():
                yaml_data = yaml.safe_load(yaml_content)
            else:
                yaml_data = {}
        except yaml.YAMLError as e:
            raise yaml.YAMLError(f"Invalid YAML frontmatter: {e}")
        
        text_data = markdown_content.strip() if markdown_content else ""
        return yaml_data, text_data
    else:
        if content.startswith('---'):
            lines = content.split('\n')
            closing_found = False
            for i, line in enumerate(lines[1:], 1):
                if line.strip() == '---':
                    closing_found = True
                    break
            
            if not closing_found:
                return None, content.strip()
        
        return None, content.strip()

def create_frontmatter(title: str, author: str = "Author", date: str = None, 
                      date_modified: str = None, categories: List[str] = None, 
                      draft: bool = False, **kwargs) -> Dict[str, Any]:
    if date is None:
        date = datetime.now().strftime('%Y-%m-%d')
    if date_modified is None:
        date_modified = datetime.now().strftime('%Y-%m-%d')
    
    base = {
        'title': title,
        'author': author,
        'date': date,
        'date-modified': date_modified,
        'draft': draft
    }
    
    if categories:
        base['categories'] = categories
    
    base.update(kwargs)
    return base

def make_quarto(path: str, yaml_data: Dict[str, Any], text_data: str = None, update_dates = True) -> None:
    """
    Make a Quarto (.qmd) document with YAML metadata and text content.
    """
    try:
        yaml_copy = yaml_data.copy()
        if update_dates: yaml_copy['date-modified'] = datetime.now().strftime('%Y-%m-%d')
        
        if text_data is None:
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    existing_content = f.read()
                
                if existing_content.startswith('---'):
                    parts = existing_content.split('---', 2)
                    if len(parts) >= 3:
                        text_data = parts[2].strip()
                    else:
                        text_data = ""
                else:
                    text_data = existing_content.strip()
            except FileNotFoundError:
                text_data = ""
        
        yaml_str = yaml.dump(yaml_copy, default_flow_style=False, sort_keys=False, allow_unicode=True)
        
        new_content = f"---\n{yaml_str}---"
        if text_data:
            new_content += f"\n\n{text_data.strip()}\n"
        
        Path(path).parent.mkdir(parents=True, exist_ok=True)
        
        with open(path, 'w', encoding='utf-8') as f:
            f.write(new_content)
            
        print(f"\nSuccessfully updated Quarto document: {path}")
        
    except Exception as e:
        print(f"Error updating Quarto document: {e}")
        raise



In [ ]:
#| export

def read_markdown_content(md_path: Path) -> str:
    """Read markdown content from an .md file."""
    try:
        with open(md_path, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except UnicodeDecodeError:
        with open(md_path, 'r', encoding='latin-1') as f:
            return f.read().strip()

def filename_to_title(filename: str) -> str:
    """Convert filename to a readable title."""
    # Remove extension and replace hyphens/underscores with spaces
    title = Path(filename).stem
    title = title.replace('-', ' ').replace('_', ' ')
    # Capitalize first letter of each word
    return ' '.join(word.capitalize() for word in title.split())

def check_listing_metadata(sync_root: Path) -> bool:
    """Check if there's a _listing_meta.yml file in the sync directory."""
    listing_file = sync_root / "_listing_meta.yml"
    return listing_file.exists()

def create_index_qmd(dir_path: Path, quarto_root: Path) -> None:
    """Create an index.qmd file for a directory."""
    dir_name = dir_path.name
    title = filename_to_title(dir_name)
    description = f"Notes on {title}"
    
    yaml_data = {
        'title': title,
        'description': description
    }
    
    # Calculate relative path to listing metadata file
    listing_file = quarto_root / "_listing_meta.yml"
    if listing_file.exists():
        # Calculate how many levels deep this directory is from quarto_root
        try:
            rel_path = dir_path.relative_to(quarto_root)
            depth = len(rel_path.parts)
            # Create the correct relative path with appropriate number of "../"
            listing_meta_path = "../" * depth + "_listing_meta.yml"
            
            yaml_data['metadata-files'] = [listing_meta_path]
            yaml_data['listing'] = {
                'contents': [
                    "/*.ipynb",
                    "/*.qmd", 
                    "/*/index.qmd"
                ]
            }
            yaml_data['margin-footer'] = "<br>"
        except ValueError:
            # dir_path is not relative to quarto_root, skip listing metadata
            pass
    
    index_path = dir_path / "index.qmd"
    make_quarto(str(index_path), yaml_data, "", update_dates=False)

def md_content_changed(md_path: Path, qmd_path: Path) -> bool:
    """Check if the markdown content has changed between .md and .qmd files."""
    if not qmd_path.exists():
        return True
        
    md_content = read_markdown_content(md_path)
    
    try:
        _, qmd_content = parse_quarto(str(qmd_path))
        return md_content != qmd_content
    except Exception:
        return True

def obsidian_to_quarto_transform(src_path: Path, dst_root: Path, sync_root: Path) -> bytes:
    """
    Transform function that converts .md files to .qmd files with proper frontmatter.
    """
    # Get relative path from source
    rel_path = src_path.relative_to(src_path.parents[len(src_path.parents) - 1])
    
    # Create corresponding .qmd path
    qmd_rel_path = rel_path.with_suffix('.qmd')
    qmd_path = dst_root / qmd_rel_path
    
    # Read markdown content
    md_content = read_markdown_content(src_path)
    
    # Check if .qmd file already exists
    if qmd_path.exists():
        # Parse existing .qmd file to preserve frontmatter
        try:
            existing_yaml, _ = parse_quarto(str(qmd_path))
            if existing_yaml:
                # Update date-modified
                existing_yaml['date-modified'] = datetime.now().strftime('%Y-%m-%d')
                yaml_data = existing_yaml
            else:
                # Create new frontmatter if none exists
                title = filename_to_title(src_path.name)
                yaml_data = create_frontmatter(title)
        except Exception:
            # If parsing fails, create new frontmatter
            title = filename_to_title(src_path.name)
            yaml_data = create_frontmatter(title)
    else:
        # Create new frontmatter for new file
        title = filename_to_title(src_path.name)
        yaml_data = create_frontmatter(title)
    
    # Create the .qmd content
    yaml_str = yaml.dump(yaml_data, default_flow_style=False, sort_keys=False, allow_unicode=True)
    qmd_content = f"---\n{yaml_str}---"
    if md_content:
        qmd_content += f"\n\n{md_content}\n"
    
    return qmd_content.encode('utf-8')



In [ ]:
#| export

def sync_obsidian_to_quarto(
    obsidian_root: str | Path,
    quarto_root: str | Path,
    dry_run: bool = True,
    ignore: Iterable[str] = ("*.DS_Store", "Thumbs.db", ".obsidian", "*.tmp")
) -> None:
    """
    Sync Obsidian .md files to Quarto .qmd files.
    
    Args:
        obsidian_root: Path to Obsidian vault or folder with .md files
        quarto_root: Path to Quarto project destination
        dry_run: If True, only show what would be done without making changes
        ignore: Glob patterns to ignore during sync
    """
    obsidian_root = Path(obsidian_root).resolve()
    quarto_root = Path(quarto_root).resolve()

    cln_quarto =  ('/').join(str(quarto_root).split('/')[-2:]) + "/"
    cln_obsidian = ('/').join(str(obsidian_root).split('/')[-2:]) + "/"
    
    print(f"Syncing from {cln_obsidian} to {cln_quarto}")
    print("-" * 50)
    
    # Check for listing metadata
    has_listing_meta = check_listing_metadata(quarto_root)
    if has_listing_meta:
        print(f"Found listing metadata file: _listing_meta.yml")
    
    # Find all .md files in obsidian root
    md_files = []
    directories_created = set()
    
    for md_path in obsidian_root.rglob("*.md"):
        if should_ignore(md_path, ignore):
            continue
            
        rel_path = md_path.relative_to(obsidian_root)
        qmd_rel_path = rel_path.with_suffix('.qmd')
        qmd_path = quarto_root / qmd_rel_path
        
        # Track directories that need index files
        parent_dirs = list(rel_path.parents)[:-1]  # Exclude root
        for parent_dir in parent_dirs:
            directories_created.add(parent_dir)
        
        md_files.append((md_path, qmd_path, rel_path))
    
    # Create directories and index files
    for dir_rel_path in sorted(directories_created):
        dir_path = quarto_root / dir_rel_path
        index_path = dir_path / "index.qmd"
        
        if not index_path.exists():
            print(f"\nCREATE INDEX: {dir_rel_path}/index.qmd")
            if not dry_run:
                dir_path.mkdir(parents=True, exist_ok=True)
                create_index_qmd(dir_path, quarto_root)
    
    # Process .md files
    for md_path, qmd_path, rel_path in md_files:
        qmd_rel_str = str(rel_path.with_suffix('.qmd'))
        
        if not qmd_path.exists():
            print(f"CREATE FILE: {qmd_rel_str}")
            if not dry_run:
                qmd_path.parent.mkdir(parents=True, exist_ok=True)
                data = obsidian_to_quarto_transform(md_path, quarto_root, quarto_root)
                with open(qmd_path, 'wb') as f:
                    f.write(data)
        else:
            # Check if content has changed
            if md_content_changed(md_path, qmd_path):
                print(f"UPDATE FILE: {qmd_rel_str}")
                if not dry_run:
                    data = obsidian_to_quarto_transform(md_path, quarto_root, quarto_root)
                    with open(qmd_path, 'wb') as f:
                        f.write(data)
            else:
                print(f"SKIP FILE: {qmd_rel_str} (no changes)")


In [ ]:
if __name__ == "__main__":
    # Example usage
    obsidian_path = "/Users/rehabpath/Local_Projects/note-sync/playground/Obsidian/notes-folder"
    quarto_path = "/Users/rehabpath/Local_Projects/note-sync/playground/Website/notes"
    
    # Dry run first to see what would happen
    sync_obsidian_to_quarto(obsidian_path, quarto_path,dry_run=False)
    
    # Uncomment to actually perform the sync
    # sync_obsidian_to_quarto(obsidian_path, quarto_path, dry_run=False)

Syncing from Obsidian/notes-folder/ to Website/notes/
--------------------------------------------------
Found listing metadata file: _listing_meta.yml
SKIP FILE: agents/huggingface-course/Unit 1 - Introduction.qmd (no changes)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()